In [1]:
import sys
sys.path.append("../Modules/")
sys.path.append("../")
import os

import pandas as pd
import numpy as np

from configobj import ConfigObj
from configobj import flatten_errors
from validate import Validator
    
import logging

from analysisParameters import loadParameters
from loadDataE1 import loadSessionDict
import processDataE1 as pF

#fmt = 'logging.Formatter(''%(levelname)s_%(name)s-%(funcName)s(): - %(message)s'
fmt = '%(levelname)s_%(name)s-%(funcName)s(): - %(message)s'
logging.basicConfig(level=logging.INFO, format=fmt)
logger = logging.getLogger(__name__)



In [2]:
# sessionDict = projectTrajectories(sessionDict)
import matplotlib.pyplot as plt
%matplotlib notebook

In [3]:
subNum = 7
bNum = 2
tNum = 128


In [4]:
fileTimeList = [];
[fileTimeList.append(name) for name in os.listdir("../Data/") if name[0] is not '.'];
fileTimeList

from analysisParameters import loadParameters
from loadDataE1 import loadSessionDict
from processDataE1 import projectTrajectories

fileName = fileTimeList[subNum]
analysisParameters = loadParameters(fileName)

# set loadProcessedData to True if you want to reprocess the file.  
sessionDict = loadSessionDict(analysisParameters,
                                  loadParsedData = True,
                                  loadProcessedData = True)

sessionDict['processed']['subjectNumber'] = subNum
sessionDict['processed']['fileName'] = fileName


INFO_loadDataE1-loadSessionDict(): - Loading preprocessed data for exp_data-2018-4-27-13-41 ***


In [5]:
from processDataE1 import gazeAnalysisWindow
sessionDict = gazeAnalysisWindow(sessionDict, analyzeUntilXSToArrival =  .3, 
                                 stopAtXSToArrival = 0.1)


INFO_processDataE1-gazeAnalysisWindow(): - Added sessionDict['trialInfo']['analysisStartFr']
INFO_processDataE1-gazeAnalysisWindow(): - Added sessionDict['trialInfo']['analysisEndFr']


In [6]:
tr = sessionDict['processed'].groupby(('blockNumber','trialNumber')).get_group((bNum,tNum))
trInfo = sessionDict['trialInfo'].groupby(('blockNumber','trialNumber')).get_group((bNum,tNum))

### Parameters

In [7]:
winSizeMS = 20
polyorder = 3
interpResS = .001

### Vector movement model

# Interpolate trial data

In [8]:
winStartTimeMs = analysisParameters['analysisWindowStart']
winEndTimeMs = analysisParameters['analysisWindowEnd']

trialTime_fr = np.array(tr['frameTime'],np.float) - np.array(tr['frameTime'],np.float)[0]
interpTime_s = np.arange(0,trialTime_fr[-1],interpResS)

# intepolate ball and gaze Az / El
ballRadiusDegs_s = np.interp(interpTime_s,trialTime_fr,np.array(tr['ballRadiusDegs'],dtype=np.float))

interpBallAz_s = np.interp(interpTime_s,trialTime_fr,np.array(tr['ball_az'],dtype=np.float))
interpBallEl_s = np.interp(interpTime_s,trialTime_fr,np.array(tr['ball_el'],dtype=np.float))

interpGazeAz_s = np.interp(interpTime_s,trialTime_fr,np.array(tr['cycGIW_az'],dtype=np.float))
interpGazeEl_s = np.interp(interpTime_s,trialTime_fr,np.array(tr['cycGIW_el'],dtype=np.float))

# To do:  plot this velocity vs raw velocity for sanity check    
# ( I had the change parameters for the s-g filter)
cycToBallVelAz_s = savgol_filter(interpBallAz_s, winSizeSamples, polyorder, deriv=1, 
                                      delta = interpResS, axis=0, mode='constant', cval=1.0)

cycToBallVelEl_s = savgol_filter(interpBallEl_s, winSizeSamples, polyorder, deriv=1, 
                                      delta = interpResS, axis=0, mode='constant', cval=1.0)




NameError: name 'savgol_filter' is not defined

In [ ]:

distTrav_s = np.cumsum(np.sqrt( np.diff(interpBallAz_s)**2 + np.diff(interpBallEl_s)**2 ) ) 
distTrav_s = np.hstack([0, distTrav_s])

initTTC = float(trInfo['ballInitialPos','Z']) / -float(trInfo['ballInitialVel','Z'])
ttcEndSampleIdx = np.where( interpTime_s > initTTC )[0][0]-1

passingTime = trialTime_fr[int(trInfo[('passVertPlaneAtPaddleFr', '')])]
passingSampleIdx = np.where( interpTime_s > passingTime)[0][0]

# Win start/end relative to initial TTC
winStartSampleIdx = np.where( interpTime_s > initTTC + winStartTimeMs/1000.0 )[0][0]
winEndSampleIdx = np.where( interpTime_s > initTTC + winEndTimeMs/1000.0 )[0][0]

# If passing sample idx < window end, raise error
if( passingSampleIdx < winEndSampleIdx):
    logger.warn('Ball collision occurs within analysis window!')

# Find where the ball / gaze actually ended up
ballWinStart_AzEl = [interpBallAz_s[winStartSampleIdx], interpBallEl_s[winStartSampleIdx]]
ballWinEnd_AzEl = [interpBallAz_s[winEndSampleIdx], interpBallEl_s[winEndSampleIdx]]
gazeWinStart_AzEl = [interpGazeAz_s[winStartSampleIdx], interpGazeEl_s[winStartSampleIdx]]
gazeWinEnd_AzEl = [interpGazeAz_s[winEndSampleIdx], interpGazeEl_s[winEndSampleIdx]]

##  Calculate ball velocity using a savitzky-golay filter
from scipy.signal import savgol_filter
    
winSizeSamples = int( (winSizeMS/1000.) / interpResS)
if(np.mod(winSizeSamples,2)==0): 
    winSizeSamples = winSizeSamples+1
    



## Fixed path, constant speed model
winDurSeconds = interpTime_s[winEndSampleIdx] - interpTime_s[winStartSampleIdx]
ballVelAtWinStart = np.sqrt(np.sum(np.power([cycToBallVelAz_s[winStartSampleIdx],
                                   cycToBallVelEl_s[winStartSampleIdx]],2)))
ballDegsMovement = winDurSeconds * ballVelAtWinStart
distRelWinStart_s = distTrav_s - distTrav_s[winStartSampleIdx]
constVelCurvSamp =  np.where(distRelWinStart_s > ballDegsMovement)[0][0]

ballAtWinEndVelPred_AzEl = [interpBallAz_s[constVelCurvSamp], 
                            interpBallEl_s[constVelCurvSamp]]


# Project gaze onto ball trajectory
gazeMinDistLoc_AzEl = np.array([interpBallAz_s[minGazeToWinSamp], interpBallEl_s[minGazeToWinSamp]])
# Calculate distance of gaze to model locations
gazeToConstantVelDist = np.sqrt(np.sum(np.power(ballAtWinEndVelPred_AzEl - gazeMinDistLoc_AzEl,2)))

# Normalize distance along continuum from vel to actual 
d = distRelWinStart_s
modelToModelDist = d[winEndSampleIdx]-d[constVelCurvSamp]
normLocInWindow = (d[minGazeToWinSamp] - d[constVelCurvSamp] ) / (modelToModelDist)


###  Calculate distances from model centers and the edge of ball 
gazeToVelCenterDistDegs = np.sqrt(np.sum(np.power(np.array(ballAtWinEndVelPred_AzEl) - np.array(gazeWinEnd_AzEl),2)))
gazeToVelEdgeDistDegs = np.sqrt(np.sum(np.power(np.array(ballAtWinEndVelPred_AzEl) - np.array(gazeWinEnd_AzEl),2))) - ballRadiusDegs_s[constVelCurvSamp]

gazeToBallDistDegs = np.sqrt(np.sum(np.power(np.array(ballWinEnd_AzEl) - np.array(gazeWinEnd_AzEl),2)))
gazeToBallEdgeDistDegs = np.sqrt(np.sum(np.power(np.array(ballWinEnd_AzEl) - np.array(gazeWinEnd_AzEl),2))) - ballRadiusDegs_s[winEndSampleIdx]


balllRadiusVel = ballRadiusDegs_s[constVelCurvSamp]
ballRadiusWinEnd = ballRadiusDegs_s[winEndSampleIdx]


In [ ]:

print(gazeToVelCenterDistDegs)
print(gazeToVelEdgeDistDegs)
print(gazeToBallDistDegs)
print(gazeToBallEdgeDistDegs)


# Plot constant vel model & observed

In [ ]:
halfHFOVDegs = 80
analyzeUntilXSToArrival =  .2
stopAtXSToArrival = 0.075

endFr = int(trInfo['passVertPlaneAtPaddleFr'])
plotFr = tr.index[:endFr]

halfVFOVDegs = halfHFOVDegs / 1.77

p, ax = plt.subplots(1, 1,figsize=(10,10)) #sharey=True)
cList = ['r','g','b']
lineHandles = []

ballH = ax.plot(tr['ball_az'][plotFr],tr['ball_el'][plotFr],color='b',linewidth=3,alpha = 0.4)
gazeH = ax.plot(tr['cycGIW_az'][plotFr],tr['cycGIW_el'][plotFr],color='r',linewidth=3,alpha = 0.4)

startFr = int(trInfo['analysisStartFr'])
endFr = int(trInfo['analysisEndFr'])

windowFr = tr.index[startFr:endFr]
wStartFr = windowFr[0]
wEndFr = windowFr[-1]

from matplotlib import patches as pt

ax.add_patch(pt.Circle(ballAtWinEndVelPred_AzEl,radius=balllRadiusVel,
            fill=False,facecolor=None,ec='k',lw=5))

ax.add_patch(pt.Circle(ballWinEnd_AzEl,radius=ballRadiusWinEnd,
             fill=False,facecolor=None,ec='k',lw=5))

ax.plot(tr['ball_az'][windowFr],tr['ball_el'][windowFr],color='b',linewidth=5, alpha = 0.6)
ax.plot(tr['cycGIW_az'][windowFr],tr['cycGIW_el'][windowFr],color='r',linewidth=5,alpha = 0.6)

for pf in windowFr:
    xs = [tr['ball_az'][pf], tr['cycGIW_az'][pf]]
    ys = [tr['ball_el'][pf], tr['cycGIW_el'][pf]]
    ax.plot(xs,ys,color='k',linewidth=1,alpha = 0.3)

ax.axis('equal')
ax.axes.spines['top'].set_visible(False)
ax.axes.spines['right'].set_visible(False)
ax.axes.yaxis.grid(True)
ax.axes.xaxis.grid(True)
p.set_facecolor('w')

plt.xlim([-30,30])
plt.ylim([-15,35])

observedH = ax.scatter(ballWinEnd_AzEl[0],ballWinEnd_AzEl[1],c='k',s=150,marker='8')
constantVelH = ax.scatter(ballAtWinEndVelPred_AzEl[0],ballAtWinEndVelPred_AzEl[1],c='k',s=150,marker='v')
gazeLoc = ax.scatter(gazeMinDistLoc_AzEl[0],gazeMinDistLoc_AzEl[1],c='m',s=150,marker='x',lw=6)

ax.text(.01,.01,str('NormLoc: {0:.2f}').format(normLocInWindow),transform=ax.transAxes)
ax.text(.01,.03,str('Expansion gain: {}').format(float(trInfo['expansionGain'])),transform=ax.transAxes)

ax.legend([gazeLoc,
           constantVelH,
           observedH], 
          
          ['point nearest to gaze',
           'constant speed model',
           'actual displacement'])

# ax.text(.01,.01,str('VelModel: {0:.2f}').format(float(trInfo['velPredError'])),transform=ax.transAxes)
# ax.text(.01,.04,str('Observed: {0:.2f}').format(float(trInfo['observedError'])),transform=ax.transAxes)

# az = trInfo['ballWinEnd_AzEl'].values[0][0]
# el = trInfo['ballWinEnd_AzEl'].values[0][1]
# ax.text(.01,.07,str('BallWinEnd: {0:.2f} {1:.2f}').format(az,el),transform=ax.transAxes)

# az = trInfo['gazeWinEnd_AzEl'].values[0][0]
# el = trInfo['gazeWinEnd_AzEl'].values[0][1]
# ax.text(.01,.1,str('GazeWinEnd: {0:.2f} {1:.2f}').format(az,el),transform=ax.transAxes)

In [ ]:
# def gazeAnalysisWindow(sessionDict, 
#     analyzeUntilXSToArrival =  .3, 
#     stopAtXSToArrival = 0.1):
x
#     startFr_fr = []
#     endFr_fr = []

#     gbTrials = sessionDict['processed'].groupby(('blockNumber','trialNumber'))

#     for (blockNum,trNum), trialData in gbTrials:

#         trInfo = sessionDict['trialInfo'].groupby(('blockNumber','trialNumber')).get_group((blockNum,trNum))
#         endFr = int(trInfo['passVertPlaneAtPaddleFr'])

#         initTTC = (trialData[('ballInitialPos','Z')].iloc[0] / -trialData[('ballInitialVel','Z')].iloc[0])

#         if stopAtXSToArrival == False:

#             stopAtXSToArrival = trialData['noExpansionForLastXSeconds'].iloc[1]
#         #noExpTimeS = trialData['noExpansionForLastXSeconds'].iloc[1]

#         expStopsFr = np.where( np.cumsum(trialData['frameTime'][1:].diff()) > initTTC-stopAtXSToArrival)[0][0]
#         endFr = np.min([expStopsFr,endFr])

#         timeToArrival_fr = trialData['frameTime'] - trialData['frameTime'].iloc[endFr]
#         startFr = np.where(timeToArrival_fr>-analyzeUntilXSToArrival)[0][0]

#         startFr_fr.append(startFr)
#         endFr_fr.append(endFr)

#     sessionDict['trialInfo']['analysisStartFr'] = startFr_fr
#     sessionDict['trialInfo']['analysisEndFr'] = endFr_fr

#     logger.info('Added sessionDict[\'trialInfo\'][\'analysisStartFr\']')
#     logger.info('Added sessionDict[\'trialInfo\'][\'analysisEndFr\']')

#     return sessionDict
